In [5]:
from unittest import mock
import pytest
import requests

class HealthcheckError(Exception):
    pass

def healthcheck():
    try:
        r = requests.get('https://python.org')
    except IOError:
        pass
    else:
        if r.status == 200:
            return 'Python website is still available' in r.content
    raise HealthcheckError('Something bad happened')
    
def get_fake_get(status_code, content):
    m = mock.Mock()
    m.status_code = status_code
    m.content = content
    
    def fake_get(url):
        return m

    return fake_get
    
def raise_get(url):
    raise IOError(f'Unable to fetch url {url}')
    
@mock.patch('requests.get', get_fake_get(200, 'The official home of the Python Programming Language.'))
def test_healthcheck_success():
    assert healthcheck() is True
  
@mock.patch('requests.get', get_fake_get(200, 'Welcome to nginx!'))
def test_healthcheck_failure():
    assert healthcheck() is False
    
@mock.patch('requests.get', get_fake_get(500, 'Oops'))
def test_healthcheck_bad_status_code():
    with pytest.raises(HealthcheckError):
        healthcheck()
        
@mock.patch('requests.get', raise_get)
def test_ioerror():
    with pytest.raises(HealthcheckError):
        healthcheck()